In [1]:
import multiprocessing
import sys
from pettingzoo.test import parallel_api_test
from citylearn import GridLearn
from citylearn import MyEnv
from pathlib import Path
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3 import PPO
import gym
import numpy as np
import supersuit as ss
from pathlib import Path
from copy import copy, deepcopy
import time

In [3]:
climate_zone = 1
data_path = Path("../citylearn/data/Climate_Zone_"+str(climate_zone))
buildings_states_actions = '../citylearn/buildings_state_action_space.json'

config = {
    "data_path":data_path,
    "climate_zone":climate_zone,
    "buildings_states_actions_file":buildings_states_actions,
    "hourly_timesteps":4,
    "max_num_houses":4
}

grid = GridLearn(**config)

nenvs = 2
envs = [MyEnv(grid), MyEnv(grid)]

print('padding action/observation spaces...')
envs = [ss.pad_action_space_v0(env) for env in envs]
envs = [ss.pad_observations_v0(env) for env in envs]

print('creating pettingzoo env...')
envs = [ss.pettingzoo_env_to_vec_env_v0(env) for env in envs]

print('stacking vec env...')
nenvs = 2
envs = [ss.concat_vec_envs_v0(env, nenvs, num_cpus=1, base_class='stable_baselines3') for env in envs]

../citylearn/buildings_state_action_space.json


/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:508: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:508: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:508: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/D

padding action/observation spaces...
creating pettingzoo env...
stacking vec env...


In [5]:
grids = [grid]
for _ in range(nenvs-1):
    grids += [deepcopy(grid)]

print('setting the grid...')
for env in envs:
    for n in range(nenvs):
        env.venv.vec_envs[n].par_env.aec_env.env.env.env.env.grid = grids[n]

models = []
for m in range(nenvs):
    models += [PPO(MlpPolicy, env, verbose=2, gamma=0.999, batch_size=2, n_steps=1, ent_coef=0.01, learning_rate=0.00025, vf_coef=0.5, max_grad_norm=0.5, gae_lambda=0.95, n_epochs=4, clip_range=0.2, clip_range_vf=1) for env in envs]

setting the grid...
Using cpu device
Using cpu device
Using cpu device
Using cpu device


In [7]:
for m in range(nenvs):
    models[m].save(f"test_{m}")

In [8]:
for m in range(nenvs):
    models[m].load(f"test_{m}")

In [ ]:
# reset the models to test them
obss = [team.reset() for team in teams]
for ts in range(5): # test on 5 timesteps
    for m in range(len(models)): # again, alternate through models

        # get the current observation from the perspective of the active team
        # this can probably be cleaned up
        foo = []
        for e in range(nenvs):
            bar = list(teams[m].venv.vec_envs[n].par_env.aec_env.env.env.env.env.state().values())
            for item in bar:
                while len(item) < 29:
                    item = np.append(item, 0)
            foo += bar

        foo = np.vstack(foo)
        obss[m] = np.vstack(foo)

        action = models[m].predict(obss[m])[0] # send it to the SB model to select an action
        obss[m], reward, done, info = teams[m].step(action) # update environment